In [2]:
from tensorflow.keras.layers import Input, Dense, Flatten, Activation, Lambda
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose, Reshape
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

In [32]:
IMG_WIDTH, IMG_HEIGHT = 224,224
import matplotlib.pyplot as plt
%matplotlib inline

def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_png(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  img_resize = tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])
  return img_resize

def process_train(file_path, show_img=False):
  img = tf.io.read_file(file_path)  
  img = decode_img(img)
  if show_img:
    plt.imshow(img)
    plt.show()
    
  return img

data = pd.read_csv('../data/more_label-FullLabel.csv')
render = data['render']
url = data['url']

In [31]:

input_dir = '/home/yujie6cs/render/'
count = 0
x = np.ndarray([len(render), 224, 224, 3])
import logging
import re
pattern = re.compile(r'\d+')

for id in np.arange(len(render)):
    count += 1
    file_name = 'render' + pattern.findall(render[id])[0] + '.png'
    this_x = process_train(os.path.join(input_dir, file_name))
    # this_x = this_x.tolist()
    # x.append(this_x)
    x[id] = this_x
    if count % 1000 == 0:
        print('{} has loaded...'.format(count))
print('load {} renders done'.format(count))

KeyboardInterrupt: 

In [5]:
cate = ['media_introduction', 'others', 'location', 'social_media_profile', 'encyclopedia', \
            'qa_forum', 'shopping_item', 'list', 'media_player', 'article']
y_train = data[cate]

print(x.shape, y_train.shape)

(4927, 224, 224, 3) (4927, 10)


In [7]:
from tensorflow.keras import regularizers

def GetModel():
    input_img = Input(shape=(224, 224, 3))

    x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same', strides=(2, 2))(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same', strides=(2, 2))(x)
    x = Conv2D(4, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same', strides=(2, 2))(x)
    x = Conv2D(2, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same', strides=(2, 2))(x)
    x = Conv2D(1, (3, 3), activation='relu', padding='same')(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    output = Dense(10, activation='softmax')(x)
    
    model = Model(input_img, output)
    model.compile(optimizer=tf.keras.optimizers.Adam(0.0001),
              loss='categorical_crossentropy', metrics=['acc']) # may add f1_score later
    return model
    

img_clf = GetModel()
img_clf.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 224, 224, 16)      448       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 112, 112, 8)       1160      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 56, 56, 8)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 56, 56, 4)         292       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 28, 28, 4)         0   

In [8]:
img_clf.fit(x, y_train, batch_size=256, epochs=20, verbose=1,
                        shuffle=True, validation_split=0.20)
img_clf.save('../saved_model/img_clf.h5')

Train on 3941 samples, validate on 986 samples
Epoch 1/20
3941/3941 [==============================] - 29s 7ms/sample - loss: 2.2975 - acc: 0.5537 - val_loss: 2.2717 - val_acc: 0.9026
Epoch 2/20
3941/3941 [==============================] - 28s 7ms/sample - loss: 2.2565 - acc: 0.7549 - val_loss: 2.2005 - val_acc: 0.9432
Epoch 3/20
3941/3941 [==============================] - 28s 7ms/sample - loss: 2.1737 - acc: 0.7790 - val_loss: 2.0475 - val_acc: 0.9483
Epoch 4/20
3941/3941 [==============================] - 29s 7ms/sample - loss: 1.9986 - acc: 0.7851 - val_loss: 1.7320 - val_acc: 0.9483
Epoch 5/20
3941/3941 [==============================] - 28s 7ms/sample - loss: 1.6831 - acc: 0.7853 - val_loss: 1.2053 - val_acc: 0.9483
Epoch 6/20
3941/3941 [==============================] - 28s 7ms/sample - loss: 1.4026 - acc: 0.7853 - val_loss: 0.9092 - val_acc: 0.9483
Epoch 7/20
3941/3941 [==============================] - 29s 7ms/sample - loss: 1.3502 - acc: 0.7869 - val_loss: 0.8989 - val_acc: 0

In [18]:
img_clf = tf.keras.models.load_model('../saved_model/img_clf.h5')


(1, 224, 224, 3)


In [49]:
def Predict(img_num, img_clf, show_img=True):
    file_name = 'render' + str(img_num) + '.png'
    this_x = process_train(os.path.join(input_dir, file_name), show_img=show_img)
    t1 = np.ndarray([1,224,224,3])
    t1[0] = this_x
    max_id = np.argmax(img_clf.predict(t1))
    # print('Predicting category of ' + url[img_num])
    if not cate[max_id] == 'article':
        print('This render is most likely to be ' + cate[max_id])

for i in np.arange(900, 1100):
    Predict(i, img_clf, False)

This render is most likely to be qa_forum
This render is most likely to be qa_forum
This render is most likely to be qa_forum


NotFoundError: /home/yujie6cs/render/render933.png; No such file or directory [Op:ReadFile]